In [1]:
import numpy as np
import gpkit
from gpkit import Model, Variable
from gpkit.constraints.bounded import Bounded

​

In [47]:
#define a module describing the water
class Material():
    def __init__(self, name, rho, sigma_yield, sigma_ultimate):
        self.name = name
        self.rho  = rho
        self.sigma_yield = sigma_yield
        self.sigma_ultimate = sigma_ultimate
        
    def __str__(self):
        string = f"Material: {self.name}"
        
        return string


class Water(Model):
    def setup(self):
        rho = Variable("\\rho", 1000, "kg/m^3", "Water density")
        m   = self.m = Variable("m", "g", "Mass of water")
        V   = self.V = Variable("V", "L", "Volume of water")
        
        V_min = 0.35*gpkit.ureg.L
        
        return [m == rho*V, V >= V_min]
    
class GeneralTank(Model):
    def setup(self, P, material):
        
        #assumes the P is the maximum operating pressure 
        constraints = []
        
        
        #design
        r = self.r = Variable("r", "cm", "Tank Radius")
        t = self.t = Variable("t", "mm", "Tank Thickness")
        l = self.l = Variable("l", "cm", "Tank Length")
        
        self.V_internal = np.pi*r**2*l
        constraints += [r <= 5*gpkit.ureg.cm]
        constraints += [l <= 10*gpkit.ureg.cm]
        constraints += [r >= 10*t] #ensures thin wall approx can be used. 
        
        #pressures
        FOSY = Variable("FOSY", 1.0, "", "Factor of Safety: Yield")
        FOSU = Variable("FOSU", 1.0, "", "Factor of Safety: Ultimate")
        jb   = Variable("j_{burst}", 1.5, "", "burst factor")
        jp   = Variable("j_{proof}", 1.25, "", "proof factor")
        
        P_burst = self.P_burst = Variable("P_{burst}", "bar", "burst pressure")
        P_proof = self.P_proof = Variable("P_{proof}", "bar", "proof pressure")
        
        
        constraints += [P_burst >= jb*(1+FOSU)*P]
        constraints += [P_proof >= jp*(1+FOSY)*P]
        
        #stresses
        sigma_vm_p = self.sigma_vm_p = Variable("\\sigma_{vm, proof}", "MPa", "Proof Von misses stress in tank")
        sigma_vm_b = self.sigma_vm_b = Variable("\\sigma_{vm, burst}", "MPa", "Burst Von misses stress in tank")
        
        constraints += [sigma_vm_p == (np.sqrt(3)/2)*P_proof*r/t]
        constraints += [sigma_vm_b == (np.sqrt(3)/2)*P_burst*r/t]
        
        constraints += [sigma_vm_p <= material.sigma_yield]
        constraints += [sigma_vm_b <= material.sigma_ultimate]
        
        #mass
        m = self.m = Variable("m", "g", "Mass of tank")
        
        rho = self.rho = material.rho
        
        #mass calculator
        constraints += [m >= rho*(2*np.pi*r*t*l + 2*np.pi*r**2*t)]
        
        return constraints

    
class EllipsoidTank(Model):
    def setup(self, P, material):
        
        #assumes the P is the maximum operating pressure 
        constraints = []
        
        #assumptions
        ew = Variable("ew", 1, "", "Weld efficiency")
        
        #stresses
        sigma_vm_p = self.sigma_vm_p = Variable("\\sigma_{vm, proof}", "MPa", "Proof Von Misses stress in tank")
        sigma_vm_b = self.sigma_vm_b = Variable("\\sigma_{vm, burst}", "MPa", "Burst Von Misses stress in tank")
        sigma_vm   = self.sigma_vm   = Variable("\\sigma_{vm, max}", "MPa", "Maximum allowed Von Misses stress in tank")
        
        #calc max allowed von misses stress in tank
        constraints += [sigma_vm_p == material.sigma_yield]
        constraints += [sigma_vm_b == material.sigma_ultimate]
        constraints += [sigma_vm <= sigma_vm_p/1.1, sigma_vm <= sigma_vm_b/1.25]
        
        
        #design
        l = self.l = Variable("l", "cm", "Tank Cylinder Length")
        a = self.a = Variable("a", "cm", "Tank Cylinder Radius")
        k = self.k = Variable("k", "", "Tank Shape Factor")
        b = self.b = Variable("b", "cm", "Tank Ellipsoid depth")
        te = self.te = Variable("t_e", "mm", "Tank Ellipsoid Thickness")
        tc = self.tc = Variable("t_c", "mm", "Tank Cylinder Thickness")
        
        constraints += [k>=1, k<=2, k == a/b, a >= 10*tc, k*a>= 10*te] #definitions and limits
        constraints += [l + 2*b <= 10*gpkit.ureg.cm] #geometric constraints on this problem
        constraints += [a <= 5*gpkit.ureg.cm]
        
        #geometric factors
        #Ep = Variable("Ep", "", "Surface Area parameter")
        #constraints += [Ep >= 2.75826 + 1.0838*k + 0.150079*k**2, Ep <= 2.75826 + 1.0838*k + 0.150079*k**2]
        Ep = 2.75826 + 1.0838*k + 0.150079*k**2
        #K  = Variable("K", "", "Envelope Stress parameter")
        #constraints += [K == 0.52*k + 0.16]
        K = 0.52*k + 0.16
        
        #thicknesses
        constraints += [te >= P*a*(K+k/2)/(2*sigma_vm)]
        constraints += [tc >= P*a/(sigma_vm*ew)]
        
        
        #determine interal volume
        #V_internal = self.V_internal = Variable("V_{internal}", "cm^3", "Tank Internal Volume")
        V_internal = self.V_internal = 4*np.pi*a**3/(3*k) + 2*np.pi*a**2*l
        
        #determine tank mass
        m = self.m = Variable("m", "g", "Mass of tank")
        rho = self.rho = material.rho
        constraints += [m >= rho*(np.pi*a**2*te*Ep/k + 2*np.pi*a*l*tc)]
        
        return constraints



class MainTank(Model):
    def setup(self):
        
        #define operation condition
        P_expected = self.P_expected = Variable("P_{expected}", 4, "bar", "Maximum expected pressure")
        T = self.T = Variable("T", 300, "K", "Temperature of Main Tank")
        
        rho_al = Variable("\\rho_{Al}", 2.7,"g/cm^3","Al tank density")
        sigma_yield = Variable("\\sigma_{y, al}", 276, "MPa", "Al yield strength")
        sigma_ultimate = Variable("\\sigma_{u, al}", 310, "MPa", "Al ultimate strength")
        al = Material('Al', rho=rho_al, sigma_yield=sigma_yield, sigma_ultimate = sigma_ultimate)
        #define tank material
        self.material = al
        
        #create tank
        tank = self.tank = EllipsoidTank(P_expected, material = self.material)
        V_H2O  = self.V_H2O   = Variable("V_{H2O Tank}", "cm^3", "Volume of Water Tank")
        
        #create water
        water = Water()
        #assemble components
        components = self.components = [water, tank]
        
        constraints = []        
        constraints += components
        
        #require water to fit in tank
        constraints += [water.V <= V_H2O]
        constraints += [V_H2O <= tank.V_internal]
        
        #determine total mass
        m = self.m = Variable("m", "g", "Main Tank Mass")
        
        constraints += [m >= sum(comp.m for comp in components)]
        
        return constraints
    
    
class Helium(Model):
    def setup(self, P, V, m, T):
        #describes the ideal gas law of helium
        constraints = []
        #gas constants
        R = 8.31446261815324*(gpkit.ureg.J/gpkit.ureg.K)/gpkit.ureg.mol
        M = 4.002602*gpkit.ureg.g/gpkit.ureg.mol

        self.m = m
        constraints += [P*V == m*R*T/M]

        return constraints
    
class HeliumTank(Model):
    def setup(self):
        
        constraints = []
        components = self.components = []
        
        #create tank
        P_He_i = self.P_He_i = Variable("P_{He, initial}", "bar", "Initial maximum expected pressure of Helium Tank")
        V_He   = self.V_He   = Variable("V_{He, Tank}", "cm^3", "Volume of Helium Tank")
        
        rho_al = Variable("\\rho_{Al}", 2.7,"g/cm^3","Al tank density")
        sigma_yield = Variable("\\sigma_{y, al}", 276, "MPa", "Al yield strength")
        sigma_ultimate = Variable("\\sigma_{u, al}", 310, "MPa", "Al ultimate strength")
        al = Material('Al', rho=rho_al, sigma_yield=sigma_yield, sigma_ultimate = sigma_ultimate)
        
        tank = self.tank = EllipsoidTank(P_He_i, material = al)
        components += [tank]
        
        constraints += [tank.V_internal >= V_He]
        
        #create helium
        m_He = self.m_He = Variable("m_{He}", "g", "Mass of Helium")
        T = self.T = Variable("T", "K", "Helium Temperature")
        
        #constraints += [m_He >= 1*gpkit.ureg.g] #define minimum mass of Helium
        constraints += [T >= 100*gpkit.ureg.K, T <= 400*gpkit.ureg.K]
        
        He_initial = Helium(P = P_He_i, V = V_He, m = m_He, T = T)
        
        self.He_initial = He_initial

        components += [He_initial]
        
        #determine total mass
        m = self.m = Variable("m", "g", "He Tank Mass")
        
        constraints += [m >= sum(comp.m for comp in components)]
        
        constraints += components
        
        return constraints
        
        

In [48]:
class TankSystem(Model):
    def setup(self):
        
        constraints = []
        
        mainTank = self.mainTank = MainTank()
        heliumTank = self.heliumTank = HeliumTank()
        
        components = self.components = [mainTank, heliumTank]
        constraints += [components]
        
        #create final helium, but dont include in mass breakdown
        He_final = Helium(P = mainTank.P_expected, V = mainTank.V_H2O, m = heliumTank.m_He, T = mainTank.T)
        
        constraints += [He_final]
        #constraints += [He_initial.m >= 1*gpkit.ureg.g]

        
        m = self.m = Variable("m", "g", "mass of full tank system")
        constraints += [m >= sum(comp.m for comp in components)]
        
        return constraints

In [49]:
with gpkit.SignomialsEnabled():
    ts=TankSystem()
    
problem = Model(ts.m, ts)

print(problem.localsolve(verbosity=2,iteration_limit = 30).table())

Beginning signomial solve.
Using solver 'cvxopt'
Solving for 30 variables.
Solving took 0.0674 seconds.
Using solver 'cvxopt'
Solving for 30 variables.
Solving took 0.0628 seconds.
Solving took 2 GP solves and 0.144 seconds.

Cost
----
 352.7 [g]

Free Variables
--------------
                   | TankSystem.11
                 m : 352.7    [g]   mass of full tank system

                   | TankSystem.11/HeliumTank
   P_{He, initial} : 33.57    [bar] Initial maximum expected pressure of Helium Tank
                 T : 100      [K]   Helium Temperature
      V_{He, Tank} : 13.9     [cm³] Volume of Helium Tank
                 m : 0.8184   [g]   He Tank Mass
            m_{He} : 0.2247   [g]   Mass of Helium

                   | TankSystem.11/HeliumTank/EllipsoidTank
\sigma_{vm, burst} : 310      [MPa] Burst Von Misses stress in tank
  \sigma_{vm, max} : 248      [MPa] Maximum allowed Von Misses stress in tank
\sigma_{vm, proof} : 276      [MPa] Proof Von Misses stress in tank
      

In [53]:
#this model includes the additional constraints that the wall thicknesses must be greater than some minimum

with gpkit.SignomialsEnabled():
    ts=TankSystem()
    
with gpkit.SignomialsEnabled():
    t_min = 0.1*gpkit.ureg.mm
    P_He_max = 20*gpkit.ureg.bar
    
    addConstraint = []
    
    addConstraint += [ts.heliumTank.tank.tc>=t_min]
    addConstraint += [ts.heliumTank.tank.te>=t_min]
    addConstraint += [ts.mainTank.tank.tc>=t_min]
    addConstraint += [ts.mainTank.tank.te>=t_min]
    addConstraint += [ts.heliumTank.P_He_i<=P_He_max]

    
problem = Model(ts.m, [ts, addConstraint])


In [54]:

print(problem.localsolve(verbosity=1,iteration_limit = 30).table())

Beginning signomial solve.
Solving took 4 GP solves and 0.363 seconds.

Cost
----
 355.8 [g]

Free Variables
--------------
                   | TankSystem.13
                 m : 355.8   [g]   mass of full tank system

                   | TankSystem.13/HeliumTank
   P_{He, initial} : 20      [bar] Initial maximum expected pressure of Helium Tank
                 T : 100     [K]   Helium Temperature
      V_{He, Tank} : 23.33   [cm³] Volume of Helium Tank
                 m : 1.009   [g]   He Tank Mass
            m_{He} : 0.2247  [g]   Mass of Helium

                   | TankSystem.13/HeliumTank/EllipsoidTank
\sigma_{vm, burst} : 310     [MPa] Burst Von Misses stress in tank
  \sigma_{vm, max} : 248     [MPa] Maximum allowed Von Misses stress in tank
\sigma_{vm, proof} : 276     [MPa] Proof Von Misses stress in tank
                 a : 1.181   [cm]  Tank Cylinder Radius
                 b : 0.6205  [cm]  Tank Ellipsoid depth
                 k : 1.903         Tank Shape Factor
    